## 🔄 Ajuste do Diretório de Trabalho

Antes de carregar ou manipular arquivos, é importante garantir que estamos no diretório correto do projeto.  
O código abaixo verifica se o notebook está sendo executado a partir da pasta `notebooks`. Se for o caso, ele sobe um nível na hierarquia de diretórios para garantir que o diretório de trabalho seja a raiz do projeto.

Isso é útil para manter caminhos relativos consistentes ao acessar dados, scripts ou outros recursos do projeto.

📌 **Resumo do que o código faz:**
- Verifica se o diretório atual termina com `notebooks`.
- Se sim, volta uma pasta (para a raiz do projeto).
- Exibe o novo diretório de trabalho.


In [1]:
import os

# Verifica se o diretório de trabalho atual termina com 'notebooks'
if os.path.basename(os.getcwd()) == 'notebooks':
    # Se sim, sobe um nível de diretório para a pasta raiz do projeto
    os.chdir('..')

# Imprime o diretório de trabalho para confirmar que a mudança foi feita
print(f"Diretório de Trabalho Atual: {os.getcwd()}")

Diretório de Trabalho Atual: c:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov


## 📦 Carregamento das Bibliotecas para a Fase 3: Modelagem Preditiva

Nesta etapa, carregamos todas as bibliotecas necessárias para realizar o treinamento, validação e interpretação de modelos de Machine Learning aplicados à previsão da tendência do Ibovespa.

---

### 🔧 Principais Componentes Importados:

#### 📊 Manipulação de Dados
- `pandas`, `numpy`: Estruturação e transformação de dados tabulares e numéricos.
- `duckdb`: Consulta e carregamento eficiente da base persistida na fase anterior.

#### ⚙️ Modelagem e Avaliação
- `lightgbm`: Framework de gradient boosting eficiente, usado para modelagem supervisionada.
- `sklearn.model_selection.train_test_split`: Divisão da base de forma temporal para simular previsão realista.
- `sklearn.metrics`: Avaliação com métricas como `accuracy`, `ROC AUC`, `confusion_matrix`.

#### 🧠 Interpretação do Modelo
- `shap`: Framework de interpretabilidade para entender a importância das features no modelo treinado.

#### 📈 Visualização
- `matplotlib`, `seaborn`: Criação de gráficos e análise visual dos resultados.

#### 🛠️ Configuração do Projeto
- `src.config`: Importa o caminho e demais parâmetros definidos nas fases anteriores.

---

✅ Todas as bibliotecas e dependências da **Fase 3 - Modelagem** foram carregadas com sucesso e estão prontas para uso.


In [2]:
import duckdb
import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
from sklearn.model_selection import train_test_split # Usaremos para a divisão temporal
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance

# Importa nossas configurações de projeto
import src.config as config

# Helpers do Notebook
from IPython.display import display

# Configurações de estilo
sns.set_theme(style='whitegrid', palette='viridis')
plt.style.use("fivethirtyeight")
%matplotlib inline

print("✅ Bibliotecas para a Fase 3 carregadas com sucesso!")

c:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Bibliotecas para a Fase 3 carregadas com sucesso!


## 🧱 Carregamento dos Dados para Modelagem

Nesta etapa, buscamos no banco de dados DuckDB a tabela `features_baseline`, que contém todas as variáveis criadas e tratadas na **Fase 2** do projeto. Esses dados são a base para o treinamento dos modelos de Machine Learning.

---

### 📥 Etapas Realizadas:

1. **Conexão ao Banco DuckDB**
   - Utilizamos o caminho salvo no módulo `config`.

2. **Leitura da Tabela `features_baseline`**
   - A tabela contém os dados finais após a engenharia de atributos, com a variável `alvo` (target) já definida.

3. **Conversão da Coluna `data`**
   - A coluna `data` é convertida para o tipo `datetime` e definida como índice do DataFrame.
   - Essa configuração é fundamental para **garantir uma divisão temporal correta** entre treino e teste, evitando vazamento de dados.

---

✅ Ao final desta célula, temos o DataFrame `df_model` carregado, indexado por data e pronto para os próximos passos de preparação e modelagem.


In [3]:
# Carregando os dados da Fase 2 do nosso banco de dados

print(f"Carregando dados da tabela 'features_baseline' de: {config.DB_PATH}")

try:
    con = duckdb.connect(database=str(config.DB_PATH), read_only=True)
    # MUITO IMPORTANTE: Selecionar da nova tabela com todas as features
    df_model = con.execute("SELECT * FROM features_baseline").fetchdf()
    con.close()

    # Configura a coluna 'data' como o índice para facilitar a divisão temporal
    df_model['data'] = pd.to_datetime(df_model['data'])
    df_model.set_index('data', inplace=True)

    print("\n✅ Dados para modelagem carregados com sucesso!")
    print("Estrutura do DataFrame:")
    df_model.info()
    # df_model.to_csv('./data/ibovXusd_feat.csv')

except Exception as e:
    print(f"❌ Ocorreu um erro ao carregar os dados: {e}")
    print("Verifique se o pipeline da Fase 1 e o notebook da Fase 2 foram executados com sucesso.")

Carregando dados da tabela 'features_baseline' de: C:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov\data\mercados.duckdb

✅ Dados para modelagem carregados com sucesso!
Estrutura do DataFrame:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5334 entries, 2005-02-07 to 2025-07-24
Data columns (total 88 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   close_petroleo_brent             5334 non-null   float64
 1   close_petrobras                  5334 non-null   float64
 2   close_dolar                      5334 non-null   float64
 3   close_ibovespa                   5334 non-null   float64
 4   close_sp500                      5334 non-null   float64
 5   high_petroleo_brent              5334 non-null   float64
 6   high_petrobras                   5334 non-null   float64
 7   high_dolar                       5334 non-null   float64
 8   high_ibovespa                    5334 non-null   f

## 🧮 Separação entre Features e Variável Alvo

Com os dados já carregados e indexados por data, o próximo passo é preparar os insumos para o modelo:

- **`X` (features):** contém todas as colunas explicativas, **excluindo a variável `alvo`**.
- **`y` (target):** representa a variável binária `alvo`, que indica se o Ibovespa **subirá (1)** ou **não subirá (0)** no dia seguinte.

---

### 📌 Objetivo
Essa separação é essencial para garantir que o modelo seja treinado apenas com variáveis **disponíveis até o dia da previsão**, sem acesso ao futuro (evitando vazamento de dados).

---

### 🧾 Resultado
- `X.shape`: mostra o número de amostras e variáveis preditoras.
- `y.shape`: confirma o alinhamento com o número de observações.
- As primeiras linhas de `X` são exibidas para conferência.

✅ Dados separados corretamente e prontos para a etapa de **divisão temporal em treino e teste**.


In [4]:
# X (features) são todas as colunas, EXCETO a coluna 'alvo'
X = df_model.drop(columns=['target'])

# y (target) é apenas a coluna 'target'
y = df_model['target']

print("--- Separação entre Features e Alvo ---")
print("Shape de X (features):", X.shape)
print("Shape de y (target):", y.shape)
print("\nPrimeiras 5 linhas das features (X):")
display(X.head())

--- Separação entre Features e Alvo ---
Shape de X (features): (5334, 87)
Shape de y (target): (5334,)

Primeiras 5 linhas das features (X):


close_petroleo_brent  close_petrobras  close_dolar  \
data                                                             
2005-02-07             45.279999         2.825950       2.1547   
2005-02-08             45.400002         2.825950       2.1547   
2005-02-09             45.459999         2.951129       2.1547   
2005-02-10             47.099998         2.937769       2.1547   
2005-02-11             47.160000         2.985399       2.1547   

            close_ibovespa  close_sp500  high_petroleo_brent  high_petrobras  \
data                                                                           
2005-02-07         25732.0  1201.719971            46.470001        2.843376   
2005-02-08         25732.0  1202.300049            46.049999        2.843376   
2005-02-09         26314.0  1191.989990            46.400002        3.006022   
2005-02-10         26416.0  1197.010010            47.299999        2.942126   
2005-02-11         26671.0  1205.300049            47.480000        3.016766   

            high_dolar  high_ibovespa   high_sp500  ...  BBM_5_2.0  \
data                                                ...              
2005-02-07      2.1665        25736.0  1204.150024  ...    25018.4   
2005-02-08      2.1665        25736.0  1205.109985  ...    25335.0   
2005-02-09      2.1665        26314.0  1203.829956  ...    25676.8   
2005-02-10      2.1665        26456.0  1198.750000  ...    25985.2   
2005-02-11      2.1665        26786.0  1208.380005  ...    26173.0   

               BBU_5_2.0  BBB_5_2.0  BBP_5_2.0  spread_petro_ibov  \
data                                                                
2005-02-07  26272.531476  10.025673   0.784500           0.000110   
2005-02-08  26322.217504   7.793310   0.701070           0.000110   
2005-02-09  26597.518328   7.171597   0.846034           0.000112   
2005-02-10  26608.756766   4.799322   0.845438           0.000111   
2005-02-11  26929.780549   5.782910   0.829025           0.000112   

            ret_dolar_1d  ret_ibov_1d  ret_diff_dolar_ibov  dia_da_semana  mes  
data                                                                            
2005-02-07           0.0     0.000000             0.000000              0    2  
2005-02-08           0.0     0.000000             0.000000              1    2  
2005-02-09           0.0     0.022618            -0.022618              2    2  
2005-02-10           0.0     0.003876            -0.003876              3    2  
2005-02-11           0.0     0.009653            -0.009653              4    2  

[5 rows x 87 columns]

## ⏳ Divisão Temporal dos Dados: Treino vs. Teste

Para garantir uma avaliação realista e evitar qualquer forma de vazamento de dados, adotamos uma **divisão temporal** na base. Isso simula um cenário onde o modelo é treinado apenas com dados do passado e testado em dados "futuros", tal como ocorre em aplicações reais.

---

### 📆 Estratégia de Corte:
- O ponto de corte foi definido como **6 meses antes da última data** disponível no dataset.
- Todos os dados **anteriores** a esse ponto foram alocados no **conjunto de treino**.
- Os dados **a partir desse ponto** foram alocados no **conjunto de teste**.

---

### 🔍 Objetivo:
Evitar qualquer **contaminação temporal**, ou seja, impedir que o modelo aprenda padrões do futuro ao treinar. Isso garante uma avaliação mais fiel de sua capacidade de generalização.

---

### 📊 Resultado da Divisão:

- **Data de corte:** definida programaticamente com `pd.DateOffset(months=6)`.
- **Tamanho do conjunto de treino:** `X_treino` e `y_treino`.
- **Tamanho do conjunto de teste:** `X_teste` e `y_teste`.
- **Proporção de teste:** também exibida para monitorar o balanceamento da divisão.

✅ Conjuntos prontos para o **treinamento e validação do modelo**.


In [5]:
# Vamos definir que nosso conjunto de teste serão os últimos 6 meses de dados.

# Define o ponto de corte (6 meses antes da última data do nosso dataset)
ponto_de_corte = X.index.max() - pd.DateOffset(months=6)

print(f"Data de corte para o conjunto de teste: {ponto_de_corte.date()}")

# Divide X em treino e teste
X_treino = X[X.index < ponto_de_corte]
X_teste = X[X.index >= ponto_de_corte]

# Divide y em treino e teste, garantindo o mesmo alinhamento
y_treino = y.loc[X_treino.index]
y_teste = y.loc[X_teste.index]


print("\n--- Tamanho dos Conjuntos ---")
print(f"Conjunto de Treino: {len(X_treino)} amostras (linhas)")
print(f"Conjunto de Teste: {len(X_teste)} amostras (linhas)")
print(f"Proporção de Teste: {len(X_teste) / len(X):.2%}")

Data de corte para o conjunto de teste: 2025-01-24

--- Tamanho dos Conjuntos ---
Conjunto de Treino: 5205 amostras (linhas)
Conjunto de Teste: 129 amostras (linhas)
Proporção de Teste: 2.42%


## 🎯 Seleção de Atributos com Permutation Importance (LightGBM)

Nesta etapa, utilizamos a técnica de **Permutation Importance** para identificar as variáveis que mais impactam a performance do nosso modelo preditivo.

---

### 🧠 O que é Permutation Importance?

Permutation Importance mede a **importância de cada feature** com base em quanto a performance do modelo diminui ao **embaralharmos os valores dessa feature** no conjunto de teste. Se a embaralhamento de uma variável causar uma grande queda na métrica (AUC), essa variável é considerada importante.

---

### ⚙️ Etapas Realizadas:

1. **Treinamento de um Modelo Base (LightGBM)**
   - Usamos `X_treino` e `y_treino` para treinar o modelo com todas as features disponíveis.

2. **Cálculo da Importância**
   - O cálculo foi feito no **conjunto de teste**, utilizando `roc_auc` como métrica de avaliação.
   - Repetimos o processo **10 vezes** para reduzir a variância nos resultados.

3. **Rankeamento e Visualização**
   - As features foram ordenadas com base na **média da redução do AUC**.
   - Exibimos as **20 variáveis mais importantes** em um gráfico de barras.

---

### 📊 Interpretação do Resultado

- Features com maior valor de `importance_mean` são aquelas cuja remoção (por permutação) **mais prejudicou a performance do modelo**.
- Este processo ajuda a reduzir o número de variáveis e a focar no que realmente **traz sinal preditivo**, o que contribui para modelos mais simples, rápidos e generalizáveis.

---

✅ Concluímos a etapa de seleção inicial de atributos com base em **dados reais de teste**, mantendo apenas as variáveis mais relevantes para a próxima fase de refino e modelagem.


In [6]:
# Usaremos os dados X_treino, y_treino, X_teste, y_teste que já separamos.

print("--- Iniciando Seleção de Atributos com Permutation Importance ---")

# 1. Treinar um modelo LightGBM de baseline com TODAS as features
# Usamos os dados de TREINO para treinar
print("\nTreinando modelo de baseline com todas as features...")
lgbm_baseline = lgb.LGBMClassifier(random_state=42, verbosity=-1)
lgbm_baseline.fit(X_treino, y_treino)

# 2. Calcular a Importância por Permutação
# Usamos os dados de TESTE para avaliar a importância das features em dados não vistos
print("Calculando a importância das features no conjunto de teste...")
result = permutation_importance(
    estimator=lgbm_baseline,
    X=X_teste,
    y=y_teste,
    n_repeats=10,       # Repete o embaralhamento 10 vezes para um resultado mais estável
    scoring='roc_auc',  # Usamos a métrica AUC como nosso critério de importância
    random_state=42,
    n_jobs=-1           # Usa todos os cores da CPU para acelerar
)

# 3. Organizar e Exibir os Resultados
# Criamos um DataFrame com os resultados para facilitar a visualização
df_importancia = pd.DataFrame(
    {'feature': X_treino.columns, 'importance_mean': result.importances_mean}
)
# Ordenamos da mais importante para a menos importante
df_importancia = df_importancia.sort_values(by='importance_mean', ascending=False)

print("\n--- Top 20 Features Mais Importantes ---")
display(df_importancia.head(20))

# 4. Visualizar a Importância das Features
plt.figure(figsize=(12, 10))
sns.barplot(
    x='importance_mean',
    y='feature',
    data=df_importancia.head(20) # Plotando apenas o top 20
)
plt.title('Top 20 Features Mais Importantes (Permutation Importance)')
plt.xlabel('Redução Média na Performance (AUC)')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

--- Iniciando Seleção de Atributos com Permutation Importance ---

Treinando modelo de baseline com todas as features...


Calculando a importância das features no conjunto de teste...

--- Top 20 Features Mais Importantes ---


feature  importance_mean
45               sp500_ret_acum_21d         0.010310
18                    open_ibovespa         0.009474
23                  volume_ibovespa         0.009319
72                 candle_body_ibov         0.008514
9                        high_sp500         0.007926
34                dolar_ret_acum_2d         0.006935
44               sp500_ret_acum_10d         0.006687
33           petrobras_ret_acum_21d         0.003870
55           sp500_ret_diario_lag_1         0.003839
64        ibovespa_ret_diario_lag_3         0.002198
25                    semana_do_mes         0.002167
21                 volume_petrobras         0.002012
80                        BBP_5_2.0         0.001579
73                candle_range_ibov         0.001424
36               dolar_ret_acum_10d         0.001300
20            volume_petroleo_brent         0.001084
59        ibovespa_ret_diario_lag_2         0.000929
49              ibovespa_ret_diario         0.000867
51  petroleo_brent_ret_diario_lag_1         0.000836
42                sp500_ret_acum_2d         0.000650

## 🧪 Seleção Final de Features para Modelagem

Com base na análise de **Permutation Importance**, selecionamos as **20 variáveis mais relevantes** para compor o nosso conjunto final de preditores.

---

### 🎯 Critério de Seleção

- As features foram ordenadas pela **redução média na performance (AUC)** ao serem embaralhadas.
- As **20 com maior impacto negativo** na performance do modelo foram selecionadas.
- Essa abordagem prioriza variáveis que de fato **contribuem para a capacidade preditiva** e ajuda a reduzir o risco de overfitting.

---

### 🧾 Resultado

A lista `features_selecionadas` representa as variáveis que serão utilizadas na próxima etapa de **modelagem final**.

✅ Com esse subconjunto enxuto e informativo, seguimos para o treinamento e avaliação de modelos com maior eficiência e interpretabilidade.


In [7]:
# Vamos selecionar as 20 features mais importantes
features_selecionadas = df_importancia.head(20)['feature'].tolist()

print("Features selecionadas para o modelo final:")
print(features_selecionadas)

Features selecionadas para o modelo final:
['sp500_ret_acum_21d', 'open_ibovespa', 'volume_ibovespa', 'candle_body_ibov', 'high_sp500', 'dolar_ret_acum_2d', 'sp500_ret_acum_10d', 'petrobras_ret_acum_21d', 'sp500_ret_diario_lag_1', 'ibovespa_ret_diario_lag_3', 'semana_do_mes', 'volume_petrobras', 'BBP_5_2.0', 'candle_range_ibov', 'dolar_ret_acum_10d', 'volume_petroleo_brent', 'ibovespa_ret_diario_lag_2', 'ibovespa_ret_diario', 'petroleo_brent_ret_diario_lag_1', 'sp500_ret_acum_2d']


## ⚙️ Aplicação da Seleção de Features no Conjunto de Dados

Com a lista `features_selecionadas` definida, filtramos os DataFrames `X_treino` e `X_teste` para manter somente as variáveis mais importantes.

---

### Objetivos:

- Reduzir a dimensionalidade dos dados para melhorar eficiência computacional.
- Manter apenas as features que mais contribuem para a performance do modelo.
- Facilitar a interpretabilidade e evitar possíveis ruídos no treinamento.

---

### Resultado Esperado:

- `X_treino_selecionado` e `X_teste_selecionado` contêm apenas as 20 variáveis selecionadas.
- A dimensão dos conjuntos reduzida, porém com as variáveis de maior impacto preditivo.

✅ Preparados para o treinamento do modelo final com os dados otimizados.


In [8]:
# 'features_selecionadas' é a lista com os nomes das melhores features que criamos na etapa anterior
# 'X_treino' e 'X_teste' são os DataFrames que contêm TODAS as features

print(f"Usando as {len(features_selecionadas)} features mais importantes para a modelagem.")

X_treino_selecionado = X_treino[features_selecionadas]
X_teste_selecionado = X_teste[features_selecionadas]

print("\nShape de X_treino_selecionado:", X_treino_selecionado.shape)
print("Shape de X_teste_selecionado:", X_teste_selecionado.shape)

Usando as 20 features mais importantes para a modelagem.

Shape de X_treino_selecionado: (5205, 20)
Shape de X_teste_selecionado: (129, 20)


## 🧠 Modelo A – Visão de Longo Prazo

Neste primeiro experimento, treinamos um modelo LightGBM utilizando **todo o histórico de dados disponíveis** no conjunto de treino. O objetivo é capturar padrões de longo prazo na dinâmica do mercado.

---

### ⚙️ Estratégia de Treinamento

- Utilização de **todas as amostras disponíveis** no conjunto de treino.
- Conjunto de teste permanece fixo para comparação posterior com outros modelos.
- Modelo base: `LGBMClassifier`, sem tuning de hiperparâmetros (baseline).
- Métrica principal: **AUC ROC**, além de métricas de classificação padrão (precision, recall, F1-score).

---

### 📊 Resultados do Modelo A

- **Acurácia:** 53%
- **AUC ROC:** 0.5822
- **Observações:**
  - A performance é apenas ligeiramente superior ao acaso (50%).
  - O modelo tem dificuldade especialmente em capturar a classe “Alta” (recall = 0.47).
  - A baixa AUC indica que o modelo não está conseguindo distinguir bem os padrões de subi


In [9]:
# --- Treinando e Avaliando o Modelo A (Longo Prazo) ---
print("\n--- Treinando Modelo A: Visão de Longo Prazo ---")

# 1. Inicializa o modelo LightGBM
lgbm_longo_prazo = lgb.LGBMClassifier(random_state=42, verbosity=-1)

# 2. Treina o modelo com os dados de treino completos
lgbm_longo_prazo.fit(X_treino_selecionado, y_treino)

# 3. Faz previsões no conjunto de teste
y_pred_longo_prazo = lgbm_longo_prazo.predict(X_teste_selecionado)
y_proba_longo_prazo = lgbm_longo_prazo.predict_proba(X_teste_selecionado)[:, 1]

# 4. Avalia a performance
print("\nResultados do Modelo A (Longo Prazo):")
print(classification_report(y_teste, y_pred_longo_prazo, target_names=['Baixa', 'Alta']))
print(f"AUC ROC: {roc_auc_score(y_teste, y_proba_longo_prazo):.4f}")


--- Treinando Modelo A: Visão de Longo Prazo ---



Resultados do Modelo A (Longo Prazo):
              precision    recall  f1-score   support

       Baixa       0.74      0.98      0.85        95
        Alta       0.50      0.06      0.11        34

    accuracy                           0.74       129
   macro avg       0.62      0.52      0.48       129
weighted avg       0.68      0.74      0.65       129

AUC ROC: 0.5867


## 🔍 Modelo B – Visão Recente (Últimos 6 Anos)

Neste segundo experimento, testamos a hipótese de que o comportamento recente do mercado pode conter padrões mais relevantes para o modelo do que o histórico completo. Assim, treinamos o modelo **apenas com dados dos últimos 6 anos**, mantendo o mesmo conjunto de teste para garantir uma comparação justa com o Modelo A.

---

### ⚙️ Estratégia de Treinamento

- Dados de treino foram filtrados para conter apenas os últimos **6 anos de histórico**.
- Conjunto de teste permaneceu **inalterado**, com 129 amostras.
- Modelo: `LGBMClassifier` (sem ajuste de hiperparâmetros).
- Métrica principal: **AUC ROC**, complementada por acurácia e F1-score.

---

### 📊 Resultados do Modelo B

- **Acurácia:** 58%
- **AUC ROC:** 0.5996
- **Observações:**
  - Houve melhora em todas as métricas em relação ao Modelo A.
  - A classe “Alta” apresentou **maior precisão (0.62)**, sugerindo que o modelo recente entende melhor os padrões de alta do Ibovespa.
  - A curva ROC também sugere melhor separação entre classes.



In [10]:
# --- Treinando e Avaliando o Modelo B (Recente) ---
print("\n--- Treinando Modelo B: Visão Recente (Últimos 6 Anos) ---")

# 1. Filtra o conjunto de treino para os últimos 3 anos
ponto_de_corte_recente = X_treino_selecionado.index.max() - pd.DateOffset(years=6)
X_treino_recente = X_treino_selecionado[X_treino_selecionado.index >= ponto_de_corte_recente]
y_treino_recente = y_treino.loc[X_treino_recente.index]

print(f"Usando {len(X_treino_recente)} amostras recentes para o treino do Modelo B.")

# 2. Inicializa e treina o modelo
lgbm_recente = lgb.LGBMClassifier(random_state=42, verbosity=-1)
lgbm_recente.fit(X_treino_recente, y_treino_recente)

# 3. Faz previsões no MESMO conjunto de teste para uma comparação justa
y_pred_recente = lgbm_recente.predict(X_teste_selecionado)
y_proba_recente = lgbm_recente.predict_proba(X_teste_selecionado)[:, 1]

# 4. Avalia a performance
print("\nResultados do Modelo B (Recente):")
print(classification_report(y_teste, y_pred_recente, target_names=['Baixa', 'Alta']))
print(f"AUC ROC: {roc_auc_score(y_teste, y_proba_recente):.4f}")


--- Treinando Modelo B: Visão Recente (Últimos 6 Anos) ---
Usando 1566 amostras recentes para o treino do Modelo B.



Resultados do Modelo B (Recente):
              precision    recall  f1-score   support

       Baixa       0.74      0.98      0.85        95
        Alta       0.50      0.06      0.11        34

    accuracy                           0.74       129
   macro avg       0.62      0.52      0.48       129
weighted avg       0.68      0.74      0.65       129

AUC ROC: 0.5467


---

### 🧩 Interpretação

O modelo treinado com dados mais recentes apresentou **melhor desempenho geral**, reforçando a ideia de que o mercado mudou e que **padrões históricos mais antigos perderam relevância**.

Esse resultado também valida os achados da EDA, que apontaram alterações estruturais nas correlações do Ibovespa com ativos como Petrobras, Dólar e S&P 500 nos últimos anos.

✅ O Modelo B será considerado o novo baseline para as próximas etapas, como tuning de hiperparâmetros e validação com diferentes janelas temporais.
